### 【アーカイブ】pyocrを用いた自動集計
pytesseractに切り替える

In [19]:
import cv2
from PIL import Image
import sys
import pyocr
import pyocr.builders

In [20]:
# 文字認識
def ocr(pic):
    tools = pyocr.get_available_tools()
    if not tools:
        print("No OCR tool found")
        sys.exit(1)
    tool = tools[0]

    texts = []
    width = 275; height = 40 #切り出す画像のサイズ
    x = 675; y = 54 # 基準点

    img = cv2.imread(pic, cv2.IMREAD_GRAYSCALE)

    for _ in range(12):
        # 分割、2値化
        img_cropped = img[y:y+height, x:x+width]
        y = y + height + 12
        ret, img_thresh = cv2.threshold(img_cropped, 180, 255, cv2.THRESH_BINARY)

        if img_thresh[0][0] == 0:
            img_thresh = 255 - img_thresh

        #文字認識
        new_image = Image.fromarray(img_thresh)
        text = tool.image_to_string(
            new_image,
            lang="jpn+eng",
            builder = pyocr.builders.TextBuilder(tesseract_layout=7)
        )
        texts.append(text)

    return texts

In [21]:
# 得点計算
def calculate_score(texts, team_tag, enemy_tag):
    rank_dict = {}
    team_score, enemy_score = 0, 0
    score_array = [15, 12, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1]

    for i, text in enumerate(texts):
        if team_tag in text:
            rank_dict[i+1] = team_tag
            team_score += score_array[i]
        elif enemy_tag in text:
            rank_dict[i+1] = enemy_tag
            enemy_score += score_array[i]
        else:
            rank_dict[i+1] = "Error: " + text

    print(f"順位: {rank_dict}")
    print("獲得した合計得点")
    print(f"{team_tag}: {team_score}, {enemy_tag}: {enemy_score}")

In [22]:
texts = ocr("../data/0.jpeg")
# print(pred_array)
calculate_score(texts, "RK", "HD")

順位: {1: 'RK', 2: 'RK', 3: 'HD', 4: 'HD', 5: 'RK', 6: 'HD', 7: 'RK', 8: 'HD', 9: 'RK', 10: 'RK', 11: 'HD', 12: 'HD'}
獲得した合計得点
RK: 48, HD: 34
